In [1]:
using MaxEntChemostat2018;

INFO: Precompiling module CPLEX.
Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] uuid4(::MersenneTwister) at ./deprecated.jl:57
 [3] msg_header at /Users/Pereiro/.julia/v0.6/IJulia/src/msg.jl:18 [inlined]
 [4] msg_pub(::IJulia.Msg, ::String, ::Dict{String,String}, ::Dict{String,Any}) at /Users/Pereiro/.julia/v0.6/IJulia/src/msg.jl:30 (repeats 2 times)
 [5] send_stream(::String) at /Users/Pereiro/.julia/v0.6/IJulia/src/stdio.jl:172
 [6] send_stdio(::String) at /Users/Pereiro/.julia/v0.6/IJulia/src/stdio.jl:130
 [7] (::Base.##302#303{IJulia.#send_stderr,Timer})() at ./event.jl:436
while loading /Users/Pereiro/.julia/v0.6/COBRA/src/COBRA.jl, in expression starting on line 27
ERROR: LoadError: CPLEX not properly installed. Please run Pkg.build("CPLEX")
Stacktrace:
 [1] error(::String) at ./error.jl:21
 [2] include_from_node1(::String) at ./loading.jl:576
 [3] include(::String) at ./sysimg.jl:14
 [4] anonymous at ./<missing>:2
while loading /Users/Pereiro/.julia/v0.6

## Loading functions

In [2]:
function load_ecoli()

    sto, mets, rxns = load_ecoli_dataframes()

    # no infinities
    rxns[:lb] .= max.(rxns[:lb], -1000)
    rxns[:ub] .= min.(rxns[:ub], 1000)
    mets[:L] .= max.(mets[:L], -1000)
    mets[:V] .= min.(mets[:V], 1000)
    mets[:c] .= min.(mets[:c], 100000)

    S = sparse(sto[:i], sto[:k], sto[:s])

    return S, mets, rxns
end

function load_ecoli_dataframes(;model_data_dir = "$(pwd()[1:end-8])/ModelsData/EColi/ecoli_v2/v2.3",
                        file_prefix = "ecoli")

    sto = CSV.read("$model_data_dir/$(file_prefix)_s.csv", allowmissing=:none, types=[Int, Int, Float64, String, String]);
    mets = CSV.read("$model_data_dir/$(file_prefix)_mets.csv", allowmissing=:none, types=[String, Float64, Float64, Float64, Float64, Float64]);
#     rxns = CSV.read("$model_data_dir/$(file_prefix)_rxns.csv", allowmissing=:none, types=[String, Float64, Float64, Float64, Float64]);

    rxnspath = "/Users/Pereiro/rxns.csv";
    rxns = CSV.read(rxnspath, allowmissing=:none, types=[String, Float64, Float64, Float64, Float64]);
    
    
    return sto, mets, rxns
end
load_cho() = MaxEntChemostat2018.FBA.load_cho();

function logrange(si::Int, ei; step = 1) 
    base = collect(1:step:9)
    col = [];
    for i in si:(ei - 1)
        push!(col, ((base)*(10.0 ^ i))...);
    end
    return col;
end

logrange (generic function with 1 method)

## Testing EColi model
Como ve profe el modelo no pincha, ```model_solve``` siempre retorna ```nothing```

In [40]:
S_ecoli, mets_ecoli, rxns_ecoli = load_ecoli();
println("S $(size(S_ecoli)), mets $(size(mets_ecoli)) rxns $(size(rxns_ecoli))");

LoadError: [91mMethodError: no method matching *(::Base.TwicePrecision{Float64}, ::Base.TwicePrecision{Float64})[0m
Closest candidates are:
  *(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:424
  *(::Base.TwicePrecision, [91m::Integer[39m) at twiceprecision.jl:470
  *(::Base.TwicePrecision{R}, [91m::S<:Number[39m) where {S<:Number, R} at twiceprecision.jl:488
  ...[39m

In [52]:
costFactor  = 100000000;
rxns_ecoli[:an] .= (rxns_ecoli[:an] .^ 2) .* costFactor;
rxns_ecoli[:ap] .= (rxns_ecoli[:ap] .^ 2) .* costFactor;
# rxns_ecoli[:an][40] = rxns_ecoli[:an][40] * costFactor^2;
# rxns_ecoli[:ap][40] = rxns_ecoli[:ap][40] * costFactor^2;

In [53]:
rxns_ecoli

,id,lb,ub,ap,an
1,ACALD,-1000.0,1000.0,4.45048e-143,4.45048e-143
2,ACALDt,-1000.0,1000.0,4.45048e-143,4.45048e-143
3,ACKr,-1000.0,1000.0,4.45048e-143,4.45048e-143
4,ACONTa,-1000.0,1000.0,4.45048e-143,4.45048e-143
5,ACONTb,-1000.0,1000.0,4.45048e-143,4.45048e-143
6,ACt2r,-1000.0,1000.0,4.45048e-143,4.45048e-143
7,ADK1,-1000.0,1000.0,0.0,0.0
8,AKGDH,0.0,1000.0,4.45048e-143,4.45048e-143
9,AKGt2r,-1000.0,1000.0,4.45048e-143,4.45048e-143
10,ALCD2x,-1000.0,1000.0,4.45048e-143,4.45048e-143


In [37]:
CSV.write("/Users/Pereiro/rxns.csv", rxns_ecoli)

CSV.Sink{Void,DataType}(    CSV.Options:
        delim: ','
        quotechar: '"'
        escapechar: '\\'
        missingstring: ""
        dateformat: nothing
        decimal: '.'
        truestring: 'true'
        falsestring: 'false'
        internstrings: true, IOBuffer(data=UInt8[...], readable=true, writable=true, seekable=true, append=false, size=0, maxsize=Inf, ptr=1, mark=-1), "/Users/Pereiro/rxns.csv", 15, true, String["id", "lb", "ub", "ap", "an"], 5, false, Val{false})

In [38]:
MaxEntChemostat2018.FBA.find_max_xi(S_ecoli, mets_ecoli, rxns_ecoli)

Academic license - for non-commercial use only


LoadError: [91mAssertionError: 0 < ξ[39m

In [15]:
using Plots

In [22]:
plot(rxns_ecoli[:ap])

In [27]:
ξrange = logrange(-3,3; step = 5);
println("ξrange ($(first(ξrange)),$(last(ξrange))) length $(length(ξrange)) ");
LP = MaxEntChemostat2018.FBA.gurobi_problem(S_ecoli, mets_ecoli, rxns_ecoli);
fbaress = [begin 
#         LP = MaxEntChemostat2018.FBA.gurobi_problem(S, mets, rxns);
        return MaxEntChemostat2018.FBA.model_solve!(LP, S_ecoli, mets_ecoli, rxns_ecoli, ξ);
        end for ξ in ξrange];
for r in 1:length(ξrange)
    fbares = fbaress[r];
    if fbares != nothing
        println("$r RESULTS whit ξ = $(ξrange[r])");
        println("μ $(fbares.μ)");
        println("ϕ $(fbares.ϕ)");
        println();
    else
        println("$r NO RESULTS: fbares == nothing with ξ = $(ξrange[r])")
    end
end

ξrange (0.001,600.0) length 12 
Academic license - for non-commercial use only
1 RESULTS whit ξ = 0.001
μ 39.62209714713303
ϕ 0.0

2 RESULTS whit ξ = 0.006
μ 37.73415722349816
ϕ 0.0

3 RESULTS whit ξ = 0.01
μ 37.426022671615925
ϕ 0.0

4 RESULTS whit ξ = 0.06
μ 31.4573046758877
ϕ 0.0

5 RESULTS whit ξ = 0.1
μ 24.40118854292718
ϕ 0.0

6 RESULTS whit ξ = 0.6000000000000001
μ 5.201745634702202
ϕ 0.0

7 RESULTS whit ξ = 1.0
μ 3.4861447830462224
ϕ 0.0

8 RESULTS whit ξ = 6.0
μ 1.3416437184762486
ϕ 0.0

9 RESULTS whit ξ = 10.0
μ 1.1700836333106508
ϕ 0.0

10 RESULTS whit ξ = 60.0
μ 0.954720161533653
ϕ 0.0

11 RESULTS whit ξ = 100.0
μ 0.9374540880567201
ϕ 0.0

12 RESULTS whit ξ = 600.0
μ 0.9158714962105539
ϕ 0.0



Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] uuid4(::MersenneTwister) at ./deprecated.jl:57
 [3] msg_header at /Users/Pereiro/.julia/v0.6/IJulia/src/msg.jl:18 [inlined]
 [4] msg_pub(::IJulia.Msg, ::String, ::Dict{String,String}, ::Dict{String,Any}) at /Users/Pereiro/.julia/v0.6/IJulia/src/msg.jl:30 (repeats 2 times)
 [5] send_stream(::String) at /Users/Pereiro/.julia/v0.6/IJulia/src/stdio.jl:172
 [6] send_stdio(::String) at /Users/Pereiro/.julia/v0.6/IJulia/src/stdio.jl:130
 [7] (::Base.##302#303{IJulia.#send_stdout,Timer})() at ./event.jl:436
while loading In[27], in expression starting on line 8


## Testing Cho model

In [9]:
S_cho, mets_cho, rxns_cho = load_cho();
println("S $(size(S_cho)), mets $(size(mets_cho)) rxns $(size(rxns_cho))");

S (2054, 3578), mets (2054, 6) rxns (3578, 5)


In [10]:
findmax(rxns_cho[:an])

(1936.431623931624, 918)

In [11]:
rxns_cho[:id][918]

"ETHAK"

In [12]:
ξrange = logrange(-3,3; step = 5);
println("ξrange ($(first(ξrange)),$(last(ξrange))) length $(length(ξrange)) ");
LP = MaxEntChemostat2018.FBA.gurobi_problem(S_cho, mets_cho, rxns_cho);
fbaress = [begin 
#         LP = MaxEntChemostat2018.FBA.gurobi_problem(S, mets, rxns);
        return MaxEntChemostat2018.FBA.model_solve!(LP, S, mets, rxns, ξ);
        end for ξ in ξrange];
for r in 1:length(ξrange)
    fbares = fbaress[r];
    if fbares != nothing
        println("$r RESULTS whit ξ = $(ξrange[r])");
        println("μ $(fbares.μ)");
        println("ϕ $(fbares.ϕ)");
        println();
    else
        println("$r NO RESULTS: fbares == nothing with ξ = $(ξrange[r])")
    end
end

ξrange (0.001,600.0) length 12 
Academic license - for non-commercial use only


LoadError: [91mUndefVarError: S not defined[39m